In [10]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import scipy
import scipy.io
import matplotlib.pyplot as plt
import random
import pickle
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
from scipy.stats import pearsonr

ModuleNotFoundError: No module named 'numpy'

In [13]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1 = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2 = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3 = np.delete(data_glove_3, 3, 1)

ecog_1 = raw['train_ecog'][0][0]
ecog_2 = raw['train_ecog'][1][0]
ecog_3 = raw['train_ecog'][2][0]

labels_1 = np.argmax(data_glove_1, axis=1)
labels_2 = np.argmax(data_glove_2, axis=1)
labels_3 = np.argmax(data_glove_3, axis=1)

In [8]:
def filter_data(raw_eeg, fs=1000):
    """
    Write a filter function to clean underlying data.
    Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
    Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
    distorting the underlying data!

    Input: 
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
    Output: 
        clean_data (samples x channels): the filtered signal
    """
    # dim = 100
    # b = sig.firwin(numtaps=dim + 1, cutoff=[0.15, 200], pass_zero='bandpass', fs=fs)
    b, a = sig.butter(N=2, Wn=[0.15, 200], btype='bandpass', fs=fs, output='ba')

    filtered_eeg = sig.filtfilt(b, a, x=raw_eeg, axis=0)
    
    return filtered_eeg

In [15]:
ecog_1 = filter_data(ecog_1)
ecog_2 = filter_data(ecog_2)
ecog_3 = filter_data(ecog_3)

train_test_ratio = 0.7

ecog_1_train = ecog_1[:int(train_test_ratio * ecog_1.shape[0])]
ecog_1_test = ecog_1[int(train_test_ratio * ecog_1.shape[0]):]
data_glove_1_train = data_glove_1[:int(train_test_ratio * data_glove_1.shape[0])]
data_glove_1_test = data_glove_1[int(train_test_ratio * data_glove_1.shape[0]):]

ecog_2_train = ecog_2[:int(train_test_ratio * ecog_2.shape[0])]
ecog_2_test = ecog_2[int(train_test_ratio * ecog_2.shape[0]):]
data_glove_2_train = data_glove_2[:int(train_test_ratio * data_glove_2.shape[0])]
data_glove_2_test = data_glove_2[int(train_test_ratio * data_glove_2.shape[0]):]

ecog_3_train = ecog_3[:int(train_test_ratio * ecog_3.shape[0])]
ecog_3_test = ecog_3[int(train_test_ratio * ecog_3.shape[0]):]
data_glove_3_train = data_glove_3[:int(train_test_ratio * data_glove_3.shape[0])]
data_glove_3_test = data_glove_3[int(train_test_ratio * data_glove_3.shape[0]):]

In [22]:
data_glove_1_train.reshape(-1, 200, 4).mean(axis=1).shape

(420, 4)

In [242]:
# Arrange ecog_1_train into 2000 points windows with stride 500
ecog_1_train_new = np.array([ecog_1_train[i:i+2000] for i in range(0, ecog_1_train.shape[0] - 2000, 500)])
data_glove_1_train_new = np.array([data_glove_1_train[i:i+2000] for i in range(0, data_glove_1_train.shape[0] - 2000, 500)])


ecog_1_valid_new = np.array([ecog_1_test[i:i+2000] for i in range(2000, ecog_1_test.shape[0] - 2000, 500)])
data_glove_1_valid_new = np.array([data_glove_1_test[i:i+2000] for i in range(2000, data_glove_1_test.shape[0] - 2000, 500)])
data_glove_1_valid_new.shape

(172, 2000, 4)

In [319]:
class FingerDataset(Dataset):
    def __init__(self, ecog, dg, window=2000):
        self.ecog = np.float32(ecog.reshape(-1, 1, window, ecog.shape[1]))
        self.ecog = (self.ecog - self.ecog.mean()) / self.ecog.std()
        self.dg = np.float32(dg.reshape(-1, window, 4).mean(axis=1))

    def __len__(self):
        return len(self.ecog)

    def __getitem__(self, idx):
        # r = r[:, np.random.permutation(r.shape[1])]
        r = self.ecog[idx]
        r = r[:,:, np.random.permutation(r.shape[2])]
        return r, self.dg[idx]

In [273]:
class FingerDataset(Dataset):
    def __init__(self, ecog, dg, window=2000):
        self.ecog = np.float32(ecog.reshape(ecog.shape[0], 1, ecog.shape[1], ecog.shape[2]))
        self.ecog = (self.ecog - self.ecog.mean()) / self.ecog.std()
        self.dg = np.float32(dg.mean(axis=1)).argmax(axis=1)

    def __len__(self):
        return len(self.ecog)

    def __getitem__(self, idx):
        return self.ecog[idx], self.dg[idx]

In [320]:
dataset_s1_train = FingerDataset(ecog_1_train.copy(), data_glove_1_train.copy())
dataset_s1_valid = FingerDataset(ecog_1_test.copy(), data_glove_1_test.copy())

train_loader = DataLoader(dataset_s1_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s1_valid, batch_size=16, shuffle=False)

In [328]:
dataset_s1_train[3][0]

array([[[-0.28872958, -0.39639145,  0.18533479, ..., -0.09954405,
          0.4653647 , -0.2981479 ],
        [-0.3409584 , -0.4518811 ,  0.11538914, ..., -0.14991836,
          0.38839257, -0.2993777 ],
        [-0.41199115, -0.51223487,  0.0390866 , ..., -0.21089442,
          0.3024675 , -0.2975337 ],
        ...,
        [ 0.65357035, -0.36680967, -0.1312886 , ..., -0.23567398,
          0.06565088, -0.2011317 ],
        [ 0.65955096, -0.3942174 , -0.13500713, ..., -0.2441146 ,
          0.04929653, -0.21758616],
        [ 0.66925377, -0.42243156, -0.13053615, ..., -0.24147059,
          0.03689422, -0.22789973]]], dtype=float32)

In [323]:
r = np.array([[  1,  20, 100],
       [  2,  31, 401],
       [  8,  11, 108]])

r = r[:, np.random.permutation(r.shape[1])]
r

array([[  1, 100,  20],
       [  2, 401,  31],
       [  8, 108,  11]])

In [287]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1,62), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        #self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.conv2 = nn.Conv2d(1, 4, (8, 8))
        self.batchnorm2 = nn.BatchNorm2d(4)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 144 timepoints. 
        #4, 2, 487
        self.fc1 = nn.Linear(132, 64) #4,1,10
        self.fc2 = nn.Linear(64, 4) #4,1,10
        

    def forward(self, x):
        # Layer 1
      #  x = x.double()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.5)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.5)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        # print (x.size())
        # FC Layer
        x = torch.flatten(x, start_dim=1)# 4*2*9) # for T=128
        #x = x.view(-1,4 *2 * 9)# 4*2*9) # for T=128
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [329]:
from torch.nn.modules.module import _addindent


class Conv2dWithConstraint(nn.Conv2d):
    def __init__(self, *args, max_norm=1, **kwargs):
        self.max_norm = max_norm
        super(Conv2dWithConstraint, self).__init__(*args, **kwargs)

    def forward(self, x):
        self.weight.data = torch.renorm(
            self.weight.data, p=2, dim=0, maxnorm=self.max_norm
        )
        return super(Conv2dWithConstraint, self).forward(x)


class EEGNet(nn.Module):
    def InitialBlocks(self, dropoutRate, *args, **kwargs):
        block1 = nn.Sequential(
            nn.Conv2d(1, self.F1, (1, self.kernelLength), stride=1, padding=(0, self.kernelLength // 2), bias=False),
            nn.BatchNorm2d(self.F1, momentum=0.01, affine=True, eps=1e-3),

            # DepthwiseConv2D =======================
            Conv2dWithConstraint(self.F1, self.F1 * self.D, (self.channels, 1), max_norm=1, stride=1, padding=(0, 0),
                                 groups=self.F1, bias=False),
            # ========================================

            nn.BatchNorm2d(self.F1 * self.D, momentum=0.01, affine=True, eps=1e-3),
            nn.ELU(),
            nn.AvgPool2d((1, 4), stride=4),
            nn.Dropout(p=dropoutRate))
        block2 = nn.Sequential(
            # SeparableConv2D =======================
            nn.Conv2d(self.F1 * self.D, self.F1 * self.D, (1, self.kernelLength2), stride=1,
                      padding=(0, self.kernelLength2 // 2), bias=False, groups=self.F1 * self.D),
            nn.Conv2d(self.F1 * self.D, self.F2, 1, padding=(0, 0), groups=1, bias=False, stride=1),
            # ========================================

            nn.BatchNorm2d(self.F2, momentum=0.01, affine=True, eps=1e-3),
            nn.ELU(),
            nn.AvgPool2d((1, 8), stride=8),
            nn.Dropout(p=dropoutRate))
        return nn.Sequential(block1, block2)


    def ClassifierBlock(self, inputSize, n_classes):
        return nn.Sequential(
            nn.Linear(1952, n_classes, bias=False),
            nn.Softmax(dim=1))

    def CalculateOutSize(self, model, channels, samples):
        '''
        Calculate the output based on input size.
        model is from nn.Module and inputSize is a array.
        '''
        data = torch.rand(1, 1, channels, samples)
        model.eval()
        out = model(data).shape
        return out[2:]

    def __init__(self, n_classes=4, channels=60, samples=151,
                 dropoutRate=0.5, kernelLength=64, kernelLength2=16, F1=8,
                 D=2, F2=16):
        super(EEGNet, self).__init__()
        self.F1 = F1
        self.F2 = F2
        self.D = D
        self.samples = samples
        self.n_classes = n_classes
        self.channels = channels
        self.kernelLength = kernelLength
        self.kernelLength2 = kernelLength2
        self.dropoutRate = dropoutRate

        self.blocks = self.InitialBlocks(dropoutRate)
        self.blockOutputSize = self.CalculateOutSize(self.blocks, channels, samples)
        self.classifierBlock = self.ClassifierBlock(self.F2 * self.blockOutputSize[1], n_classes)

    def forward(self, x):
        x = self.blocks(x)
        x = x.view(x.size()[0], -1)  # Flatten
        x = self.classifierBlock(x)

        return x

def categorical_cross_entropy(y_pred, y_true):
    # y_pred = y_pred.cuda()
    # y_true = y_true.cuda()
    y_pred = torch.clamp(y_pred, 1e-9, 1 - 1e-9)
    return -(y_true * torch.log(y_pred)).sum(dim=1).mean()

def torch_summarize(model, show_weights=True, show_parameters=True):
    """Summarizes torch model by showing trainable parameters and weights."""
    tmpstr = model.__class__.__name__ + ' (\n'
    for key, module in model._modules.items():
        # if it contains layers let call it recursively to get params and weights
        if type(module) in [
            torch.nn.modules.container.Container,
            torch.nn.modules.container.Sequential
        ]:
            modstr = torch_summarize(module)
        else:
            modstr = module.__repr__()
        modstr = _addindent(modstr, 2)

        params = sum([np.prod(p.size()) for p in module.parameters()])
        weights = tuple([tuple(p.size()) for p in module.parameters()])

        tmpstr += '  (' + key + '): ' + modstr
        if show_weights:
            tmpstr += ', weights={}'.format(weights)
        if show_parameters:
            tmpstr +=  ', parameters={}'.format(params)
        tmpstr += '\n'

    tmpstr = tmpstr + ')'
    return tmpstr

In [333]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1,62), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        # self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        # self.conv3 = nn.Conv2d(4, 4, (8, 4))
        # self.batchnorm3 = nn.BatchNorm2d(4, False)
        # self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        #4, 2, 487
        self.fc1 = nn.Linear(8016, 256)
        self.fc2 = nn.Linear(256, 4)
        

    def forward(self, x):
        # Layer 1
      #  x = x.double()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        # x = self.padding2(x)
        # x = F.elu(self.conv3(x))
        # x = self.batchnorm3(x)
        # x = F.dropout(x, 0.25)
        # x = self.pooling3(x)
        # print (x.size())
        # FC Layer
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = EEGNet()#.cuda(0)
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))#.cuda(0))))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), weight_decay=0.01, lr=0.001)

In [334]:
net = EEGNet() #.cuda(0)
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))#.cuda(0))))
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.01)

net.train()
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        output = net(ecog)
        pred += [output.detach().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # _, predicted = torch.max(output.data, 1)
        #     # print(predicted, dg)
        # total += dg.shape[0]
        # correct += (predicted == dg).sum().item()
    pred = np.concatenate(pred)
    train_cor = pearsonr(np.repeat(pred, 2000, axis=0)[:,0], data_glove_1_train[:,0]).statistic
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total

    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            output = net(ecog)
            pred += [output.detach().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()
            # _, predicted = torch.max(output.data, 1)
            # print(predicted, dg)
            # total += dg.shape[0]
            # correct += (predicted == dg).sum().item()
        pred = np.concatenate(pred)
        val_cor = pearsonr(np.repeat(pred, 2000, axis=0)[:,0], data_glove_1_test[:,0]).statistic
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 21.505 | Train Cor: -0.087 | Valid loss: 2.971 | Valid Cor: -0.05184911596521971
Epoch 2 | Train loss: 1.949 | Train Cor: 0.055 | Valid loss: 0.799 | Valid Cor: -0.01102028937899473
Epoch 3 | Train loss: 0.778 | Train Cor: -0.121 | Valid loss: 0.669 | Valid Cor: 0.027125290769804113
Epoch 4 | Train loss: 0.577 | Train Cor: 0.119 | Valid loss: 0.619 | Valid Cor: -0.0037869722863958233
Epoch 5 | Train loss: 0.512 | Train Cor: 0.047 | Valid loss: 0.570 | Valid Cor: 0.058404663872196325
Epoch 6 | Train loss: 0.456 | Train Cor: 0.259 | Valid loss: 0.530 | Valid Cor: 0.055863685162193935
Epoch 7 | Train loss: 0.439 | Train Cor: 0.177 | Valid loss: 0.526 | Valid Cor: 0.1364335562479656
Epoch 8 | Train loss: 0.439 | Train Cor: 0.211 | Valid loss: 0.543 | Valid Cor: 0.05547492846314258
Epoch 9 | Train loss: 0.428 | Train Cor: 0.213 | Valid loss: 0.528 | Valid Cor: 0.06287463884047424
Epoch 10 | Train loss: 0.426 | Train Cor: 0.137 | Valid loss: 0.565 | Valid Cor: 0.0252012

KeyboardInterrupt: 

In [223]:
torch.max(output.data, 1)[1] == dg


tensor([ True, False, False, False, False, False, False,  True, False, False,
        False, False, False, False,  True, False])

In [71]:
pred.shape

(180, 4)

In [69]:
np.repeat(pred, 500, axis=0).shape

(90000, 4)

In [77]:
pearsonr(np.repeat(pred, 500, axis=0)[:,0], data_glove_1_test[:,0]).statistic

-0.0491939338496332